In [1]:
#PERMANENT CELL 1

import os
import mikeio
import mikeio1d
from mikeio1d.res1d import Res1D
import math
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ctypes
import traceback
MessageBox = ctypes.windll.user32.MessageBoxA
from Result_Lookup_Variables import *

In [3]:
for m in master_list:
    
#     model_area = m[0]
#     model = m[1]
    result_folder = m[2]
#     output_folder = m[3]
    result_list = m[4]    
    
    for r in result_list:
        header = r[0]
        file = r[1]
        res1d = Res1D(result_folder + '\\' + file)
        r.append(res1d)


In [87]:
result_folder = r"J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\Always_Latest_Master_Model_Simulations_FSA\Model"
node_df = pd.read_csv(result_folder + '\\ls_nodes.csv',dtype={'Node': str})
node_df.set_index('Node',inplace=True)
pipe_df = pd.read_csv(result_folder + '\\ls_pipes.csv',dtype={'Pipe': str,'FromNode': str,'ToNode': str})
pipe_df.set_index('Pipe',inplace=True)

#Final product
Alternating rows:
Node, Link Upstream, Link Downstream, Node............Node

In [6]:
class Node():
    def __init__(self,node):    
        self.node = node
    def invert(self):        
        return node_df.loc[self.node,'Invert Level']
    def ground(self):        
        return node_df.loc[self.node,'Ground Level']
    def soh(self):        
        return node_df.loc[self.node,'Safe Operating Head']
    def asset_name(self):        
        return node_df.loc[self.node,'Asset Name']
    def cover(self):        
        return node_df.loc[self.node,'Cover Type']

class Pipe():
    def __init__(self,pipe):    
        self.pipe = pipe
    def height(self):        
        return pipe_df.loc[self.pipe,'Height'] 
    def uplevel(self):
        uplevel = pipe_df.loc[self.pipe,'UpLevel']
        if math.isnan(uplevel):
            node = pipe_df.loc[self.pipe,'FromNode']
            uplevel = node_df.loc[node,'Invert Level']
        return uplevel
    def dwlevel(self):
        dwlevel = pipe_df.loc[self.pipe,'DwLevel']
        if math.isnan(dwlevel):
            node = pipe_df.loc[self.pipe,'ToNode']
            dwlevel = node_df.loc[node,'Invert Level']
        return dwlevel
    

In [ ]:
Pipe('48502').uplevel()

In [9]:
r

['Nov 15 2021 Event',
 'FSA_WWF_2021-11-12_5d_2020pop_Base.res1d',
 'Nov 15 2021 Event',

In [ ]:
math.isnan(pipe_df.loc['48502','DwLevel'])

In [88]:
missing_pipes = []

acronyms = list(pipe_df.Acronym.unique())
paths = []

loop_counter = 0

for acronym in acronyms:
    print(acronym)
    
    path_no = 0
    
    pipe_index_df = pipe_df[pipe_df.Acronym==acronym].copy()
    
    while len(pipe_index_df)>1:
             

        start_pipes = []
        for index, row in pipe_index_df.iterrows():
            if row['FromNode'] not in list(pipe_index_df.ToNode):

                start_pipes.append(index)

        start_pipes

        for start_pipe in start_pipes[:1]:
            exit_path = False
            chainage = 0

            node = pipe_index_df.loc[start_pipe,'FromNode']
            pipe = start_pipe

            path = []
            point = [path_no,0,acronym,'Node',node,chainage,Node(node).invert(),
                     Node(node).ground(),Node(node).soh(),Node(node).asset_name(),Node(node).cover(),
                    Pipe(pipe).height(),Pipe(pipe).height()+Pipe(pipe).uplevel()]
            for r in result_list:
                point.append(max(r[3].query.GetNodeValues(node, "WaterLevel")))
            path.append(point)

            i = 0
            while exit_path == False and i < 1000:

                point = [path_no,i*3+1,acronym,'Pipe US',pipe,chainage,Pipe(pipe).uplevel(),
                             Node(node).ground(),Node(node).soh(),Node(node).asset_name(),Node(node).cover(),
                            Pipe(pipe).height(),Pipe(pipe).height()+Pipe(pipe).uplevel()]

                for r in result_list:
                    point.append(max(r[3].query.GetReachStartValues(pipe, "WaterLevel")))
                path.append(point)

                chainage += pipe_index_df.loc[pipe,'Length']

                node = pipe_index_df.loc[pipe,'ToNode']


                point = [path_no,i*3+1,acronym,'Pipe DS',pipe,chainage,Pipe(pipe).dwlevel(),
                             Node(node).ground(),Node(node).soh(),Node(node).asset_name(),Node(node).cover(),
                            Pipe(pipe).height(),Pipe(pipe).height()+Pipe(pipe).dwlevel()]

                for r in result_list:
                    point.append(max(r[3].query.GetReachEndValues(pipe, "WaterLevel")))

                path.append(point)

                point = [path_no,i*3+3,acronym,'Node',node,chainage,Node(node).invert(),
                             Node(node).ground(),Node(node).soh(),Node(node).asset_name(),Node(node).cover(),
                            Pipe(pipe).height(),Pipe(pipe).height()+Pipe(pipe).dwlevel()]

                path.append(point)

                pipe_index_df.drop(pipe,inplace=True)

                try:
                    pipe = pipe_index_df[pipe_index_df.FromNode==node].index[0]
                except:
                    exit_path = True   
                i += 1
                
                loop_counter += 1
                if loop_counter > 20000:
                    raise
                    
                
        paths += path
        
        path_no += 1

# paths
columns = ['Path Number','Sqn','Acronym','Type','MUID','Chainage','Invert Level','Ground Level','Safe Operating Head',
                                   'Asset Name','Cover Type','Height','Pipe Obvert']

for r in result_list:
    header = r[0]
    columns.append(header)
    

path_df=pd.DataFrame(paths,columns=columns)
       

SAT
SAC
PSM
GL2
GLC
GLK
PFM
NWF
POC
CQC
BNL
MHT
LZW
BNP
SAS
SGP
CFF
NWX
PSE
NWG
HTA
NWS
CQM
P2F
LKI
BNS
GLN
NWR
NCT
WMT
CQD
NWC
NWW
WDT
SAB
PSW
GLO
NRT
BIN
LZS
PIF
PIM
GLE
LZF
ML2
FRP
NXI
PSO
ALT
ALX
CQL
NZF
SFM
SPP
NWA
TSF
NXJ
NXG
NXF
STF
MLT
SYC
SMS
NWP
KYT
NPL
LZG
SAP
SAD
NWD
FST
PQO
NWT
HTB
LZT
NZE
NZD
PQE
NZA
NZB
NZI
GLB
WMW
NZC
BNC
NZH
NXE
PSS
BGO
MHO
NZG
NXP
TSI
VIN
WDO
NXA
PMA
SOF
QBF
GLT
BBX
NSM
NSD
NSX
M2A
RRT
MPK
BSX
RYO
NSA
MSO
NXK
BSW
NS4E1
NSS
NSF
BSI
MPR
BBL
NSP
MSW
NSR
NKR
MPY
NET
OPC
OP6
KZS
MIS
BBB
NSW
MSM
NXB
BBC
NKN
SSH
NXC
RYA
MPA
SSG
CTS
CI2
WFM
CB1
LTS
BBA
NNA
MS2W
RE2
SSS
SSC
EF3
KTZ
BYN
MRS
NS6
SSK
NSE
SSCH
SSJ
CV7
L2F
LAF
SSA
NFR
NDI
SSP
MST
SSF
N2T
VTH
SLA
LET
CZS
OSR
NS4
LAG
SST
RE1
CVS
SSD
BST
L1F
ROC
NGW
L3C
RO6
NTS
NNK
NKM
CIN
ROH
C2F
CV6
VT2F
SRC
BSE
NDR
CMP
LGL
LBG
OPA
CTO
SDD
WHR


In [89]:

pipe_index_df

,Acronym,Height,Length,UpLevel,DwLevel,FromNode,ToNode
Pipe,,,,,,,
55738,WHR,0.45,51.0,1.62,1.62,35445,Node_7703


In [71]:
pipe_df.pipe

<bound method NDFrame.pipe of       Acronym  Height      Length  UpLevel  DwLevel FromNode ToNode
Pipe                                                               
40151     SAT   0.750  120.068282      NaN      NaN     1092   7761
40152     SAC   0.450   54.500463      NaN      NaN    23088   7763
40153     SAC   0.450   35.500124      NaN      NaN     7774  23088
40154     SAC   0.450   26.010145      NaN      NaN     7775   7774
48502     PSM   0.300   96.312622      NaN     4.63     6915   6914
...       ...     ...         ...      ...      ...      ...    ...
41183     SYC   0.375   34.336492      NaN      NaN     7701   7702
41184     SYC   0.375   36.235428      NaN      NaN     7700   7701
43762     SYC   0.200   27.754939      NaN      NaN     7224   7225
43763     SYC   0.250   71.640513      NaN      NaN     7223   7224
43764     SYC   0.250   79.560258      NaN      NaN     7222   7223

[3823 rows x 7 columns]>

In [94]:
len(path_df[path_df.Type!='Node'].MUID.unique())

3783

In [95]:
len(pipe_df)

3823

In [96]:
with open(r'J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\Always_Latest_Master_Model_Simulations_FSA\Plot_Experiments\missing.mus', 'w') as f:
    f.write('msm_Link\n')
    for pipe in pipe_df.index:
        if not pipe in  path_df[path_df.Type!='Node'].MUID.unique():
            f.write(str(pipe) + '\n')
f.close()

In [ ]:
path_acronym_df.columns

In [53]:
pipe_df

,Acronym,Height,Length,UpLevel,DwLevel,FromNode,ToNode
Pipe,,,,,,,
40151,SAT,0.750,120.068282,NaN,NaN,1092,7761
40152,SAC,0.450,54.500463,NaN,NaN,23088,7763
40153,SAC,0.450,35.500124,NaN,NaN,7774,23088
40154,SAC,0.450,26.010145,NaN,NaN,7775,7774
48502,PSM,0.300,96.312622,NaN,4.63,6915,6914
...,...,...,...,...,...,...,...
41183,SYC,0.375,34.336492,NaN,NaN,7701,7702
41184,SYC,0.375,36.235428,NaN,NaN,7700,7701
43762,SYC,0.200,27.754939,NaN,NaN,7224,7225


In [135]:
colors = ['black','black','green','red','brown','orange','grey','brown','midnightblue','tan','firebrick','darkturquoise',
         'darkorchid','dimgray','darkgoldenrod','indianred','deeppink']

#Display cases: P2F, SSS
path_acronym_df = path_df[path_df.Acronym=='SSS']
fig = go.Figure()
# fig = go.Figure(go.Scatter(x=[0,1,2,0], y=[0,2,0,0], fill="toself"))
# fig = go.Figure(go.Scatter(x=[0,1,2,0,None,3,3,5,5,3], y=[0,2,0,0,None,0.5,1.5,1.5,0.5,0.5], fill="toself"))

columns=['Invert Level','Pipe Obvert','Ground Level','Safe Operating Head']
for r in result_list:
    header = r[0]
    columns.append(header)
    
for i, column in enumerate(columns):
    if i < 4:
        line_width = 4
    else:
        line_width = 2
    fig.add_trace(go.Scatter(x=path_acronym_df.Chainage, 
                             y = path_acronym_df[column],line_color=colors[i],name=column, line_width = line_width))


    fig.add_shape(type="rect",x0=500, y0=1, x1=510, y1=3,line=dict(color="RoyalBlue",width=0.5))


fig.show()

In [32]:
path_df.Acronym.unique()

array(['SAT', 'SAC', 'PSM', 'GL2', 'GLC', 'GLK', 'PFM', 'NWF', 'POC',
       'CQC', 'BNL', 'MHT', 'LZW', 'BNP', 'SAS', 'SGP', 'CFF', 'NWX',
       'PSE', 'NWG', 'HTA', 'NWS', 'CQM', 'P2F', 'LKI', 'BNS', 'GLN',
       'NWR', 'NCT', 'WMT', 'CQD', 'NWC', 'NWW', 'WDT', 'SAB', 'PSW',
       'GLO', 'NRT', 'BIN', 'LZS', 'PIF', 'PIM', 'GLE', 'LZF', 'ML2',
       'FRP', 'NXI', 'PSO', 'ALT', 'ALX', 'CQL', 'NZF', 'SFM', 'SPP',
       'NWA', 'TSF', 'NXJ', 'NXG', 'NXF', 'STF', 'MLT', 'SYC', 'SMS',
       'NWP', 'KYT', 'NPL', 'LZG', 'SAP', 'SAD', 'NWD', 'FST', 'PQO',
       'NWT', 'HTB', 'LZT', 'NZE', 'NZD', 'PQE', 'NZA', 'NZB', 'NZI',
       'GLB', 'WMW', 'NZC', 'BNC', 'NZH', 'NXE', 'PSS', 'BGO', 'MHO',
       'NZG', 'NXP', 'TSI', 'VIN', 'WDO', 'NXA', 'PMA', 'SOF', 'QBF',
       'GLT', 'BBX', 'NSM', 'NSD', 'NSX', 'M2A', 'RRT', 'MPK', 'BSX',
       'RYO', 'NSA', 'MSO', 'NXK', 'BSW', 'NS4E1', 'NSS', 'NSF', 'BSI',
       'MPR', 'BBL', 'NSP', 'MSW', 'NSR', 'NKR', 'MPY', 'NET', 'OPC',
       'OP6', 'KZS

In [99]:
path_df[path_df['Acronym']=='PSM']

,Path Number,Sqn,Acronym,Type,MUID,Chainage,Invert Level,Ground Level,Safe Operating Head,Asset Name,Cover Type,Height,Pipe Obvert,2yr 24h No Climate Change,5yr 24h No Climate Change,10yr 24h No Climate Change,Dry Weather Flow,Jan 31 2020 Event,Nov 15 2021 Event
66,0,0,PSM,Node,4667W,0.000000,2.54,5.20,3.51,NaN,2,0.5,3.04,2.542500,2.741017,3.177094,2.542500,4.948004,5.118075
67,0,1,PSM,Pipe US,40497,0.000000,2.54,5.20,3.51,NaN,2,0.5,3.04,2.542500,2.741017,3.177094,2.542500,4.948004,5.118075
68,0,1,PSM,Pipe DS,40497,49.141321,2.42,5.68,3.30,MH22,3,0.5,2.92,2.422682,2.645077,3.058782,2.422682,4.739903,4.911263
69,0,3,PSM,Node,7546,49.141321,2.42,5.68,3.30,MH22,3,0.5,2.92,NaN,NaN,NaN,NaN,NaN,NaN
70,0,4,PSM,Pipe US,40483,49.141321,2.42,5.68,3.30,MH22,3,0.5,2.92,2.422500,2.611485,3.015710,2.422500,4.664103,4.836010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,4,13,PSM,Pipe DS,40494,336.829025,2.61,5.47,3.54,MH23,3,0.5,3.11,3.130285,3.691572,3.873064,2.846173,5.006915,5.173323
202,4,15,PSM,Node,6901,336.829025,2.61,5.47,3.54,MH23,3,0.5,3.11,NaN,NaN,NaN,NaN,NaN,NaN
203,4,16,PSM,Pipe US,40495,336.829025,2.61,5.47,3.54,MH23,3,0.5,3.11,3.128191,3.689036,3.870270,2.846173,5.004189,5.170289
204,4,16,PSM,Pipe DS,40495,366.468724,2.54,5.13,3.51,MH22A,2,0.5,3.04,3.112274,3.668551,3.848396,2.843460,4.984281,5.147463


In [64]:
pipe_index_df.drop('55738')

,Acronym,Height,Length,UpLevel,DwLevel,FromNode,ToNode
Pipe,,,,,,,


In [66]:
len(pipe_index_df.drop('55738'))

0

In [140]:
output_folder = r'J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\Always_Latest_Master_Model_Simulations_FSA\Plot_Experiments\Long_Sections'

colors = ['black','black','green','red','brown','orange','grey','brown','midnightblue','tan','firebrick','darkturquoise',
         'darkorchid','dimgray','darkgoldenrod','indianred','deeppink']


for acronym in acronyms:
    
    
    path_acronym_df = path_df[path_df.Acronym==acronym]
    
    with open(output_folder + "\\Long_Section_" + acronym + ".html", 'w') as f:
        f.write('<link rel="stylesheet" href="..\Maps_And_CSS\style.css">\n')

        f.write('<!DOCTYPE html>')
        f.write('<html>')
        f.write('<head>')
        f.write('<meta charset="utf-8">')
        f.write('</head>')
        f.write('<body>')
        f.write("<h1>Long Sections for " + acronym + "</h1>\n")
  
        for path_no in path_acronym_df['Path Number'].unique():
            path_acronym_sub_df = path_acronym_df[path_acronym_df['Path Number']==path_no]
            path_acronym_node_df = path_acronym_sub_df[path_acronym_sub_df.Type=='Node']

            fig = go.Figure()

            columns=['Invert Level','Pipe Obvert','Ground Level','Safe Operating Head']
            for r in result_list:
                header = r[0]
                columns.append(header)

            for i, column in enumerate(columns):
                if i < 4:
                    line_width = 4
                else:
                    line_width = 2
                fig.add_trace(go.Scatter(x=path_acronym_sub_df.Chainage, 
                                         y = path_acronym_sub_df[column],line_color=colors[i],name=column, line_width = line_width))

            for index, row in path_acronym_node_df.iterrows():
                
                fig.add_shape(type="rect",x0=row['Chainage']-1, y0=row['Invert Level'], x1=row['Chainage'], 
                              y1=row['Ground Level'],line=dict(color="RoyalBlue",width=1),)

            f.write("<h2>Long Section " + str(path_no+1) + "</h2>\n")
            f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
        f.write('</div>')
        f.write('</body>')
        f.write('</html>')
        f.close()

In [121]:
test = node_df.copy()
test['Shapess']='yep'
test

,Asset Name,Cover Type,Network Type,Ground Level,Invert Level,Safe Operating Head,Shapess
Node,,,,,,,
1_1129_N,,2,1,3.50,-0.46,NaN,yep
1_1130_N,,2,1,0.30,-0.46,NaN,yep
1_1131_N,,2,1,0.62,-0.14,NaN,yep
1_1132_N,,2,1,1.54,0.78,NaN,yep
1_1133_N,,2,1,1.63,0.87,NaN,yep
...,...,...,...,...,...,...,...
TimerNode_NW2Q,NaN,2,1,2.90,0.00,NaN,yep
TimerOutlet,NaN,1,3,2.00,1.50,NaN,yep
TimerOutlet2,NaN,1,3,2.00,1.50,NaN,yep
